In [27]:
import numpy as np # type: ignore
import math

In [28]:
max_base = 5e4 #in units of wavelengths
d_ant_a = 2500 #cm, size of dish
lowest_freq = 15.0 #GHz

In [29]:
def ang_res_calc(uvwave_baseline):
    """returns the angular resolution of the observation in arcseconds. uvwave_baseline should be input 
    in terms of wavelengths"""
    rad = 1/(uvwave_baseline)
    arcsec = rad * 206265
    return arcsec

In [30]:
def cell_size(angres):
    """returns the cell size in arcseconds for the image. angres input should be in arcseconds"""
    return angres/4.0

In [31]:
def highest_wave(freq_low):
    '''
    determine the highest wavelength in cm, based on the lowest frequency in GHz
    '''
    freq_hz = freq_low * 1e9
    wave = 3e10/freq_hz
    return wave

In [32]:
def fov(wavelength, baseline):
    """
    determines the fov for the image based on the highest wavelength of the obs
    in cm and the distance between the antennas in cm
    """
    rad = wavelength/baseline
    arcesc = rad*206265
    return arcesc

In [33]:
import itertools

# Generate Hamming numbers up to a high enough limit
def generate_hamming_numbers(limit):
    hamming_numbers = set()
    for a, b, c in itertools.product(range(0, 40), range(0, 30), range(0, 20)):
        num = (2 ** a) * (3 ** b) * (5 ** c)
        if num > limit * 10:  # Allow a generous upper bound
            continue
        hamming_numbers.add(num)
    return sorted(hamming_numbers)

# Return the smallest Hamming number strictly greater than `num`
def next_hamming_number_above(num):
    hamming_numbers = generate_hamming_numbers(num)
    for h in hamming_numbers:
        if h > num:
            closest = h
            break
    else:
        raise ValueError("No Hamming number found greater than the input.")

    # Factorize into powers of 2, 3, and 5
    powers = {2: 0, 3: 0, 5: 0}
    temp = closest
    for factor in [2, 3, 5]:
        while temp % factor == 0:
            powers[factor] += 1
            temp //= factor

    return closest, powers

In [34]:
def imsize(fov, cell):
    '''
    picks an image size that is the closest composite number of 2, 3, and 5 to the desired image size. fov and cell both given
    in arcesconds. returns a number of cells. 
    '''
    num_cells = fov/cell
    closest, powers = next_hamming_number_above(num_cells)
    return closest


In [35]:
angres = ang_res_calc(max_base)
cell = cell_size(angres)
wave = highest_wave(lowest_freq)
field = fov(wave,d_ant_a)
image_size = imsize(field, cell)
print(f"The angular resolution of the observation is {angres:.2f} arcsec")
print(f"The cell size for this image will be {cell:.2f} arcsec")
print(f"The image field of view will be {field:.2f} arcsec")
print(f"The image size for this image will then be {image_size:.0f} cells")

The angular resolution of the observation is 4.13 arcsec
The cell size for this image will be 1.03 arcsec
The image field of view will be 165.01 arcsec
The image size for this image will then be 162 cells
